In [1]:
import numpy as np #numerical computation
import pandas as pd #processing dataframes

Show keyboard shortcuts in Google Colab:
Ctrl/⌘ + M + H

In [2]:
# Download example data set
import urllib.request
from zipfile import ZipFile

urllib.request.urlretrieve("http://fengmai.net/download/data/bia652/pandas_data.zip", 
                           "pandas_data.zip")
ZipFile("pandas_data.zip").extractall()

# Pandas Tutorials

## Things to remember about pandas
* There are multiple (perhaps too many) ways to do simple things in pandas. Take note of the way that is most clear to you.
* Pandas functions rely heavily on the index (both row and column, especially row). This is very different from R or SAS.
* DataFrame and Series behave differently.
* Pandas functions rarely modify a dataframe in place ([see here](https://pandas.pydata.org/pandas-docs/stable/user_guide/basics.html#copying)). Most functions return a copy of the dataframe that you need to assign back to a variable.
* Pandas has evolved. Older tutorials may no longer be the best practice. Always [check the offical documentation](https://pandas.pydata.org/pandas-docs/stable/user_guide/index.html).

In [3]:
df = pd.read_csv('data/RegularSeasonCompactResults.csv')

This file identifies the game-by-game results for 31 seasons of historical NCAA basketball data, from 1985 to 2015. Each year, it includes all games played from daynum 0 through 132 (which by definition is "Selection Sunday," the day that tournament pairings are announced). Each row in the file represents a single game played.

- "season" - this is the year of the associated entry in seasons.csv (the year in which the final tournament occurs)

- "daynum" - this integer always ranges from 0 to 132, and tells you what day the game was played on. It represents an offset from the "dayzero" date in the "seasons.csv" file. For example, the first game in the file was daynum=20. Combined with the fact from the "season.csv" file that day zero was 10/29/1984, that means the first game was played 20 days later, or 11/18/1984. There are no teams that ever played more than one game on a given date, so you can use this fact if you need a unique key. In order to accomplish this uniqueness, we had to adjust one game's date. In March 2008, the SEC postseason tournament had to reschedule one game (Georgia-Kentucky) to a subsequent day, so Georgia had to actually play two games on the same day. In order to enforce this uniqueness, we moved the game date for the Georgia-Kentucky game back to its original date.

- "wteam" - this identifies the id number of the team that won the game, as listed in the "teams.csv" file. No matter whether the game was won by the home team or visiting team, "wteam" always identifies the winning team.

- "wscore" - this identifies the number of points scored by the winning team.

- "lteam" - this identifies the id number of the team that lost the game.

- "lscore" - this identifies the number of points scored by the losing team.

- "numot" - this indicates the number of overtime periods in the game, an integer 0 or higher.

- "wloc" - this identifies the "location" of the winning team. If the winning team was the home team, this value will be "H". If the winning team was the visiting team, this value will be "A". If it was played on a neutral court, then this value will be "N". Sometimes it is unclear whether the site should be considered neutral, since it is near one team's home court, or even on their court during a tournament, but for this determination we have simply used the Kenneth Massey data in its current state, where the "@" sign is either listed with the winning team, the losing team, or neither team.


## The Basics

Now that we have our dataframe in our variable df, let's look at what it contains. We can use the function **head()** to see the first couple rows of the dataframe (or the function **tail()** to see the last few rows). We can use `sample()` to look at a few random rows.

In [ ]:
df.head()

In [ ]:
df.tail()

In [ ]:
df.sample(5)

We can see the dimensions of the dataframe using the the **shape** attribute

In [ ]:
df.shape

`len()` gives the number of rows. You can also use `df.shape[0]`.

In [ ]:
df.shape[1]

In [ ]:
len(df)

We can also extract all the column names as a list, by using the **columns** attribute and can extract the rows with the **index** attribute

In [ ]:
df.columns

In [ ]:
df.columns.tolist()

In [ ]:
df.info()

In [ ]:
df['Wloc'] = df['Wloc'].astype("category") 

Should you use category or string?

In [ ]:
df['Wloc'].astype("string").nbytes

In [ ]:
df['Wloc'].astype("category").nbytes

In order to get a better idea of the type of data that we are dealing with, we can call the **describe()** function to see statistics like mean, min, etc about each column of the dataset. 

In [ ]:
df.describe()

In [ ]:
df.Wloc.describe()

In [ ]:
df.Wloc.value_counts()

Okay, so now let's looking at information that we want to extract from the dataframe. Let's say I wanted to know the max value of a certain column. The function **max()** will show you the maximum values of all columns

In [ ]:
df.count()

Then, if you'd like to specifically get the max value for a particular column, you pass in the name of the column using the bracket indexing operator

In [ ]:
df['Wscore'] #series

In [ ]:
df.Wscore

In [ ]:
df['Wscore'].max()

If you'd like to find the mean of the Losing teams' score. 

In [ ]:
df['Lscore'].mean()

But what if that's not enough? Let's say we want to actually see the game(row) where this max score happened. We can call the **idxmax()** function to identify the row index.

In [ ]:
df['Wscore'].idxmax()

One of the most useful functions that you can call on certain columns in a dataframe is the **value_counts()** function. It shows how many times each item appears in the column. This particular command shows the number of games in each season

In [ ]:
df['Season'].value_counts().head()

## Dataframe Iteration

In order to iterate through rows in a dataframe, we can use the `iterrows()` or `itertuples()` function. Note that the returned data types are different. 

**You should never modify something you are iterating over. This is not guaranteed to work.**

In [ ]:
for index, row in df.iterrows():
    print(row)
    print(type(row))
    if index == 2: # print the first 3 rows
        break

In [ ]:
for index, row in enumerate(df.itertuples()):
    print(row)
    print(type(row))
    if index == 2:
        break

## Acessing Values: [[]], loc, iloc

The bracket indexing operator is one way to extract certain columns from a dataframe.

In [ ]:
subset_df = df[['Wscore', 'Lscore']]

In [ ]:
subset_df.head()

Note the difference is the return types when you use brackets and when you use double brackets. 

In [ ]:
type(df['Wscore'])

`df.var` is a shortcut for `df[var]`.

In [ ]:
type(df.Wscore)

Double bracket returns a dataframe with 1 column.

In [ ]:
type(df[['Wscore']])

You've seen before that you can access columns through df['col name']. You can access rows by using slicing operations.   
**This is not recommended because it leads to confusion on whether you are slicing row index or row numbers.**

In [ ]:
for x in range(0, 3):
  print(x)

In [ ]:
df[0:3] #### DO NOT USE THIS NOTATION 

**It is better to be explicit and use an equivalent iloc, which slices row/col numbers.**

In [ ]:
df.iloc[0:3, :]

In [ ]:
df.iloc[0:3, [1,3]]

Iloc is definitely one of the more important functions. The main idea is that you want to use it whenever you have the integer index of a certain row that you want to access. As per Pandas documentation, iloc is an "__integer-location based indexing for selection by position.__"

The other really important function in Pandas is the **loc** function. Contrary to iloc, which is an integer based indexing, loc is a "__Purely label-location based indexer for selection by label__". Since all the games are ordered from 0 to 145288, iloc and loc are going to be pretty interchangable in this type of dataset. But __loc allows for accessing both rows and columns using labels__.

**VERY IMPORTANT**: 

**In pandas, index means label. Each row has a label, each col has a label (col names). Row index (label) may not be row numbers, they can be date, ID, time, or any other strings.**

In [ ]:
df[['Wscore', 'Lscore']].iloc[0:3, :]

In [ ]:
# row index 0-3, col index Wscore and Lscore. 
df.loc[0:3, ['Wscore', 'Lscore']]

Notice the slight difference in that iloc exlcudes the end range (behave like pure Python), while loc is inclusive (behave like database queries). 

In [11]:
df.loc[[df['Wscore'].idxmax()]] 
# Which game has the highest Wscore? Note that idxmax() returns the row *label* (not number) of the maximum value of a series. 
# A prior version of this tutorial incorrectly uses df.iloc[[df['Wscore'].idxmax()]], which leads to a mix-up of row labels and numbers.
# In our case, because row labels = row numbers the outputs are the same, 
# but in other cases it may introduces bugs that are difficult to detect. 

,Season,Daynum,Wteam,Wscore,Lteam,Lscore,Wloc,Numot
24970,1991,68,1258,186,1109,140,H,0


Each dataframe has a **values** attribute which is useful because it basically displays your dataframe in a numpy array style format

In [ ]:
df.Wscore.values

Now, you can simply just access elements like you would in an array. 

In [ ]:
%%timeit
df.values[0, 0]

In [ ]:
%%timeit
df.iloc[0,0]

In [ ]:
%%timeit
df.iat[0,0]

The latter is much faster (test using %%timeit).

If you'd like to see more discussion on how loc and iloc are different, check out this great Stack Overflow post: http://stackoverflow.com/questions/31593201/pandas-iloc-vs-ix-vs-loc-explanation. Just remember that **iloc looks at integer-based positions** and **loc looks at labels**. Loc becomes very important when your row labels aren't integers. 

### Sorting

Let's say that we want to sort the dataframe in increasing order for the scores of the losing team

In [ ]:
df.sort_values('Lscore', ascending=False).head()

### Filtering Rows Conditionally

Now, let's say we want to find all of the rows that satisy a particular condition. For example, I want to find all of the games where the winning team scored more than 180 points. The idea behind this command is you want to access the column 'Wscore' of the dataframe df (df['Wscore']). You first create a Boolean mask series (True/False) using `df['Wscore'] > 180`, and then returns only those specific rows (`df[df['Wscore'] > 180]`).

In [ ]:
df['Wscore'] > 180

In [ ]:
df[df['Wscore'] > 180]

This is equivalent to:

In [ ]:
df.query('Wscore > 180')

The df[] is a shortcut for .loc. You can combine the boolean mask with column labels.

In [ ]:
df.loc[df['Wscore'] > 180, ['Season', 'Wscore', 'Wteam', 'Lteam']]

In [ ]:
df.loc[~(df['Wscore'] > 180), ['Season', 'Wscore', 'Wteam', 'Lteam']] # NOT

This also works if you have multiple conditions. Let's say we want to find out when the winning team scores more than 150 points and when the losing team scores below 100. 

In [ ]:
df[(df['Wscore'] > 150) & (df['Lscore'] < 100)] # AND 

In [ ]:
df[(df['Wscore'] > 150) & (df['Wloc'].isin(['A','H']))] # AND 

In [ ]:
df[(df['Wscore'] > 150) | (df['Lscore'] < 100)] # OR

In [ ]:
df.query('Wscore > 150 or Wloc == "H"')

### Creating New Variables

There are multiple ways of creating new varialbes, but we will begin by using *just the indexing operator* (the brackets). Place a string inside of the brackets and make this the left-hand side of the assignment.

The right-hand side can consist of any of the following:
* A scalar value
* A list or array with the same length as the DataFrame
* A pandas Series with an index that matches the index of the DataFrame (tricky!)

**New column assigned to a scalar value**

A **scalar** value is simply one single value, like an integer, string, boolean or date. When using a scalar for column assignment, each value in the column will be the same. Let's create a column **`NO_MEANING`** and assign it the value 99.

In [ ]:
df['NO_MEANING'] = 99
df.head()

In [ ]:
df.drop(columns=['NO_MEANING'])

Instead of creating a new column with all the same values, we can use a list or NumPy array with different values for each row. The only stipulation is that the number of new values in the list/array must be the same as the number of rows in the DataFrame.

Let's create the column `BONUS RATE`, with a list of random numbers between 0 and 1.

In [ ]:
df['BONUS RATE'] = np.random.uniform(size=df.shape[0])
df.head()

In [ ]:
 np.random.uniform(size=df.shape[0])

__Warning__:  
To use a Series (e.g. from another dataframe) to create a new column, the index must match that of the modifying DataFrame. **Be very careful!!**

### Creating new values conditionally

**Warning: Be careful about updating values**
The best way to create/update values in a datafram is to assign values to the .iloc or loc querys.

Example: Create a new variable 'high_score' that takes the value of 1 if both winning and losing team scored at least 70. 

In [ ]:
df['high_score'] = 0

#### wrong approach #1

In [ ]:
high_score_games = df[(df['Wscore'] >= 70) & (df['Lscore'] >= 70)]

In [ ]:
high_score_games.head()

In [ ]:
high_score_games['high_score'] = 1 

In [ ]:
df.head()

#### wrong approach #2

In [ ]:
df[(df['Wscore'] >= 70) & (df['Lscore'] >= 70)]['high_score'] = 1 

In [ ]:
df.head()

#### correct approach #1

In [ ]:
df.loc[(df['Wscore'] >= 70) & (df['Lscore'] >= 70), 'high_score'] = 1   

In [ ]:
df.head()

#### correct approach #2

In [ ]:
df['high_score'] = 0

apply, map, applymap

In [ ]:
df['high_score'] = df.apply(lambda x: 1 if (x['Wscore'] >= 70) & (x['Lscore'] >= 70) else 0, axis = 1) 

In [ ]:
df.head()

#### correct approach #3

In [ ]:
df['high_score'] = 0
df = df.assign(high_score=lambda x: ((x['Wscore'] >= 70) & (x['Lscore'] >= 70)).astype('int'))
 # note that assign returns a new df

In [ ]:
df.head()

### Create a new column with expressions involving other columns
We can create a new column by combining any number of other columns. One primary way of doing that is through a mathematical expression. For instance, let's create a new column **`BONUS`** by multiplying a random **`BONUS RATE`** between 0, 1 and **`Wscore - Lscore`** columns together.

Note that `df['Wscore'].sub(df['Lscore'])` is more robust than `df['Wscore'] - df['Lscore']` because it handles missing values well. Similar functions include `add`, `mul`, `div`, `pow`, etc. 

In [ ]:
df['BONUS RATE'] = np.random.uniform(size = len(df))

In [ ]:
df['BONUS'] = df['BONUS RATE'] * (df['Wscore'] - df['Lscore'])
df.head()

In [ ]:
df['BONUS'] = df['BONUS RATE'] * (df['Wscore'].sub(df['Lscore']))
df.head()

Another way to create new columns is to appy a function row wise. The function should take a row as input. Apply is usually slower than the vectorised functions.

In [ ]:
df['BONUS'] = df.apply(lambda row: row['BONUS RATE'] * (row['Wscore'] - row['Lscore']), axis=1)
df.head()


def new_var(row):
  return row['BONUS RATE'] * (row['Wscore'] - row['Lscore'])

df['BONUS'] = df.apply(new_var, axis=1)


## Group and Aggregate

Another important function in Pandas is **groupby()**. This is a function that allows you to group entries by certain attributes (e.g Grouping entries by Wteam number) and then perform operations on them. The following function finds the mean Wscore and Lscore for each season. 

In [ ]:
df.groupby(['Season'])[['Wscore', 'Lscore']].mean().head()

This next command groups all the games with the same season and finds where how many times each team won. 

In [ ]:
df.groupby('Season')['Wteam'].value_counts()

The above Series has a MultiIndex (2 levels of indicies, Season and Wteam). We can use `loc` to select a year:

In [ ]:
df.groupby('Season')['Wteam'].value_counts().loc[2016]

We can use [IndexSlice](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.IndexSlice.html) to select a team (e.g., 1385):

In [ ]:
df.groupby('Season')['Wteam'].value_counts().loc[pd.IndexSlice[1990:1995, 1385]].plot()

Note that the above is a Series, we can use `reset_index` to get a dataframe

In [ ]:
df.groupby('Season')['Wteam'].value_counts().reset_index(name="Number of Wins")

We can also `rename` the variables using a dictionary.

In [ ]:
df.groupby('Season')['Wteam'].value_counts().reset_index(name="Number of Wins").rename(columns = {'Wteam':'Team', 'Season':'Year'})

Note that after grouping, the group becomes the index. So we can use `loc` function to find the top 2 high scores from 1985 to 1991.

In [ ]:
df.groupby('Season')['Wscore'].nlargest(2).loc[1985:1991]

In [ ]:
df.groupby('Season')['Wscore'].nlargest(2).loc[1985:1991].reset_index()

We can also write our own aggregate functions. Here, we want to find the percentage of games where the *loser* scored at least 80 points. 

The aggregate function takes a series as input, and returns a number as output. 

In [ ]:
def find_pct_games_gt_80(x):
    return 100*sum(x > 80)/len(x)

df.groupby('Season').Lscore.agg(find_pct_games_gt_80).loc[2010:2016]

5.13% of losing team scored more than 80 points in 2010.

We can apply the same agg function to multiple columns. It seems that 2016 is a high scoring season. 

In [ ]:
df.groupby('Season')[['Wscore', 'Lscore']].agg(find_pct_games_gt_80).loc[2010:2016]

## Long-to-wide and Wide-to-long

Take the following dataframe as an example. For each season, there are two columns of values. Sometimes we want to transform this **wide** dataframe to a **long** dataframe (key-value pairs) using `melt()`.

In [ ]:
wide_df = df.groupby(['Wteam', 'Season'])[['Wscore', 'Lscore']].mean().head().reset_index()
wide_df

In [ ]:
long_df = pd.melt(wide_df, id_vars=['Wteam', 'Season'], value_vars=['Wscore', 'Lscore'], var_name = 'W/L', value_name = 'score')
long_df

We can revert the operation and transform a long dataframe to a wide one using `pivot()`.

In [ ]:
pd.pivot_table(long_df, index = ['Wteam', 'Season'], columns = 'W/L', values = 'score').reset_index()

## Merging dataframes

The above aggregated dataframe provides the summary of when a team is winning, its socre and opponent score by season. 

In [ ]:
winning_summary = df.groupby(['Wteam', 'Season'])[['Wscore', 'Lscore']].mean().reset_index()
winning_summary

Alternatively, we want to see how each team performs when it is losing

In [ ]:
losing_summary = df.groupby(['Lteam', 'Season'])[['Wscore', 'Lscore']].mean().reset_index()
losing_summary

We want to merge the above two dataframes so we can compare each team's performance when they are losing vs winning.   
There are multiple ways to merge in pandas. The way that is most similar to SQL and other langauge is

`pd.merge(left, right, how='left', left_on=['lkey1', 'lkey2'], right_on=['rkey1', 'rkey2'], suffixes=('_left', '_right'))`

In [ ]:
pd.merge(winning_summary, losing_summary, how='inner', left_on=['Wteam', 'Season'], right_on=['Lteam', 'Season'], 
         suffixes=('_when_winning', '_when_losing')).drop(columns='Lteam').rename(columns={"Wteam": "teamID"})

Note that the above method does not depend on Index (hence we use the `reset_index()`. By default `join` can be used to join dataframes using Index. 

In [ ]:
losing_summary = df.groupby(['Lteam', 'Season'])[['Wscore', 'Lscore']].mean()
winning_summary = df.groupby(['Wteam', 'Season'])[['Wscore', 'Lscore']].mean()

In [ ]:
losing_summary

In [ ]:
winning_summary

In [ ]:
losing_summary.index.names =['team', 'Season']
winning_summary.index.names =['team', 'Season']

In [ ]:
winning_summary.join(losing_summary, how='inner', lsuffix='_when_winning', rsuffix='_when_losing')

`merge` can also use index. Therefore, there is little reason for you to use `join`.

In [ ]:
winning_summary.merge(losing_summary, how='inner', left_index=True, right_index=True, suffixes=('_when_winning', '_when_losing'))